# Quantum Pipeline

Define small feature map (angle encoding, ZZFeatureMap).

Define variational ansatz (1–3 layers).

Measure expectation values → feature vector.

Train classical classifier on these features.

In [ ]:
import pennylane as qml
import pennylane.numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc
import joblib

In [3]:
data = np.load("data/mnist01_pca4.npz")    #from dataprep notebook

X_train, y_train = data["X_train"], data["y_train"]  
X_test, y_test = data["X_test"], data["y_test"]

# x = features/ dimensions --> 4
# y = labels
n_qubits=4
dev= qml.device("default.qubit", wires=n_qubits)

print("PCA reduced shape xtrain:  ", X_train.shape)
print("PCA reduced shape xtest:  ", X_test.shape)
print("ytrain labels: ", y_train.shape)
print("ytest labels: ",y_test.shape)


PCA reduced shape xtrain:   (12665, 4)
PCA reduced shape xtest:   (2115, 4)
ytrain labels:  (12665,)
ytest labels:  (2115,)


## Quantum Feature Map

Classical data is transformed into quantum states. For the main pipeline, I will be using PennyLane as it is easier to build custom circuits and I can compare feature maps in a single framework. Pennylane also supports Qiskit backeneds for a future plugin to run on IBM devices. 


<br>

This is a quantum map $\phi(\mathbf{x})$ from classical feature vector $\mathbf{x}$ to quantum state $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$ in the Hilbert space, achieved through applying the parameterized unitary operator $\mathcal{U}_{\Phi(\mathbf{x})}$ to the initial state $|0\rangle^{n}$, n being the num of qubits for encoding. This is done so quantum algorithms can use the data. 

Unitary operations are a building block of Qcircuits, where a matrix U is unitary if $$U^\dagger U = U U^\dagger = I$$ 

Some examples are the Pauli X Gate or Hadamard Gate, or in this case, the Rotation Gate $R_y(\theta)$. 
$$
R_y(\theta) = \begin{bmatrix}\cos(\theta/2) & -\sin(\theta/2) \\ \sin(\theta/2) & \cos(\theta/2)\end{bmatrix}, \quad
R_y^\dagger(\theta) = R_y(-\theta), \quad
R_y(\theta) R_y^\dagger(\theta) = I
$$


### ZZ Map Manual - PennyLane

For a ZZ Feature Map, PennyLane does not already have a built in function so we have to do it manually using Hadamard gates, along wiht RZ and ZZ rotations for each qubit pair. This just makes it simpler to integrate over the course of this project. The Qiskit version of this gate is shown in my quantum testing ground. 

Also, out of all encoding methods I could've picked, Angle Encoding seemed to be the most simple and effective when working with circuits (perfect when I am working on a simple project).

The ZZ feature map is a version of the Pauli feature map, which includes entangling gates (ZZ interactions thorugh controlled-phase gates), done after initial data encoding rotations. This is done to allow feature map to capture any potential correlations between features and then maps the data to a more complex hilbert space. 

There are layers of Hadamard gates, single qubit phase rotations, and 2 qubit controlled phase rotations. 

<br>

Hadamard Gates

Creates equal superposition of either state within a qubit

$$
H = \tfrac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1 \\[6pt]
1 & -1
\end{pmatrix}
$$

RZ Gates 

Single qubit phase rotation around Z axis, diagonal gate

$$
R_Z(\phi) 
= \exp\!\left(-i \tfrac{\phi}{2} Z\right) 
= 
\begin{pmatrix}
e^{-i\phi/2} & 0 \\[6pt]
0 & e^{\,i\phi/2}
\end{pmatrix}
$$


CNOT Gates

Flips target qubit if control qubit is 1.  --> encodes pairwise correlations

$$
\text{CNOT} = CX =
\begin{pmatrix}
1 & 0 & 0 & 0 \\[6pt]
0 & 1 & 0 & 0 \\[6pt]
0 & 0 & 0 & 1 \\[6pt]
0 & 0 & 1 & 0
\end{pmatrix}
$$

<br>

ZZ Feature Map

Encodes the pairwise correlations between features (by constructing a unitary transformation)

$$
U_{\mathrm{ZZ}}(x) = \exp\left( i \sum_i x_i Z_i + i \sum_{i<j} x_i x_j Z_i Z_j \right)
$$


In [4]:
sample_vec = X_train[0]      #1st of 12665
print("Example feature vector: ", sample_vec)  


Example feature vector:  [-4.39083685  1.26077612  1.578084    0.85940065]


In [5]:
def zz_feature_map(x, wires, reps=1):      #(classical data vector, qubits in use, number of layers/repititions)
    n_qubits = len(wires)

    for _ in range(reps):
        for i in wires: 
            qml.Hadamard(wires=i)            #hadamard gates, uniform superposition of states --> gives room for inference in future entanglement 

        for i, wire in enumerate(wires):     #single qubit phase rotations (encodes classical data to qubit phases)
            qml.RZ(2*x[i], wires=wire)       #RZ rotation, angle = 2x[i] 

        for i in range(n_qubits-1):                    #ZZ entangling layer for pairs of neighboring qubits/wires
            qml.CNOT(wires=[wires[i], wires[i+1]])
            qml.RZ(2*x[i]*x[i+1], wires=wires[i+1])    #performs rotation around Z axis by given amount (angle = 2x[i] * x[i+1]) ENCODES PAIRWISE CORRELATIONS BETWEEN FEATURES x[i] and x[i+1]
            qml.CNOT(wires=[wires[i], wires[i+1]])     #uncomputes entanglement (so final effect is ZZ interaction)


# he
@qml.qnode(dev)
def qnode_measure(x):
    zz_feature_map(x, wires=range(4), reps=1)
    return qml.state()


print(f"ZZFeatureMap state vector: \n {qnode_measure(sample_vec)} \n") 
print(qml.draw(qnode_measure, level='device')(sample_vec))    


ZZFeatureMap state vector: 
 [-0.24166348+0.06402159j  0.12856535+0.21440838j  0.24664338+0.04082946j
  0.16880032-0.18440839j -0.02931574-0.24827522j -0.23039082+0.09705706j
  0.24496536+0.04991964j  0.17550227-0.17804199j -0.07578422+0.23823675j
  0.24991017+0.00670117j  0.16690489-0.18612565j -0.06489921-0.24142927j
  0.20609065+0.14151552j  0.07877125-0.23726587j -0.19894346+0.15139848j
  0.01831538+0.24932819j] 

0: ──H──RZ(-8.78)─╭●─────────────╭●─────────────────────────────────┤ ╭State
1: ──H──RZ(2.52)──╰X──RZ(-11.07)─╰X─╭●───────────╭●─────────────────┤ ├State
2: ──H──RZ(3.16)────────────────────╰X──RZ(3.98)─╰X─╭●───────────╭●─┤ ├State
3: ──H──RZ(1.72)────────────────────────────────────╰X──RZ(2.71)─╰X─┤ ╰State


In this circuit, you can see how each qubit has a Hadamard gate applied (without this, qubits starting in |0⟩ and RZ would only add a global phase - unobservable, as RZ works with relative phase differences between |0⟩ and |1⟩). 

Then, a phase rotation of (x) amount around the Z axis to encode the classical features x[i] as a Z axis phase shift of angle 2x[i].  

Finally, we have the entanglement layers that work through each adjacent pair of qubits to perform phase shifts. Applies controlled phase depending on both qubits and encodes pairwaise correlations into the state. 

**Creates a nonlienar feature map in Hilbert space.**

A CNOT sandwich, which is seen within the multiqubit entanglement layer above, allows for conditional application of a targets rotation, depending on the control qubit. So, in the end, we are maintaining the computational values of the qubit but keeping the phase entanglement of the qubits (it would look the same on the 0,1 level but now hidden quantum phase correlations encode the data)

## Variational Ansatz 

Ansatz: basic architecture of circuit, set of gates that act on specific subsystems with a few assumptions about the appropriate training circuit. Can be generic/problem neutral (hardware ansatz) or can be problem specific. 

Quantum Ansatz: Parameterized Quantum Circuit (PQC) that represents a family of possible quantum stats that are controlled by tunable paramters (normally angles)

$$
|\psi(\theta)\rangle = U(\theta) |0\rangle
$$

ansatz circuit: $$U(\theta)$$ parameters: $$\theta = [\theta_1, \theta_2, \ldots]$$ 


Method used to approimate ground state (lowest energy eingenstate) of quantum system by selecting a trial wavefunction with adjustable parameters. The expectation value of the energy will always be >= to the true ground state energy, so if we optimize the trial wavefunction, we can minimize the expectation value. Built from rotation gates (paramterized by angles) and entangling gates --> these form a layer, we can form many and stakc them to increase expresssiveness. 

**Analogy to CC**

Similar to how a NN layer has weights = rotational angles in the quantum gates. 
- training adjusts the angles to minimize some error/loss 
- patterns to how the gates connect = circuits architecture 

Also similar to deeper NN layers, we can stack them to increase expressiveness


In [11]:
def var_ansatz(params): #params = layers, n_qubits, 3 
    for l in range(params.shape[0]):     #loops through all layers
        for i in range(n_qubits):     #single qubit rotations 
            qml.Rot(params[l,i,0], params[l,i,1], params[l,i,2], wires=i)     # paramertized rotations
        
        for i in range(n_qubits-1):     #entanglers for non linear relationships 
            qml.CNOT(wires=[i, i+1])     #[i:control, i+1:target]


#intial params tensors - random, will be trained in ansatz 
params = np.random.randn(2, n_qubits, 3)     # 2 layers, n_qubits =4, 3 euler angles for each rotation axis 

@qml.qnode(dev)
def ansatz_preview(params):
    var_ansatz(params)
    return qml.state()

print(qml.draw(ansatz_preview)(params))



0: ──Rot(1.41,0.18,0.65)───╭●──Rot(0.58,-0.63,0.49)───────────────────────╭●──────────────────── ···
1: ──Rot(0.74,-1.69,-0.43)─╰X─╭●─────────────────────Rot(0.83,-0.01,1.05)─╰X──────────────────── ···
2: ──Rot(0.59,0.91,-0.33)─────╰X────────────────────╭●─────────────────────Rot(-0.57,0.77,-0.90) ···
3: ──Rot(0.61,0.02,0.03)────────────────────────────╰X─────────────────────Rot(-0.15,1.31,0.90)─ ···

0: ··· ───────┤  State
1: ··· ─╭●────┤  State
2: ··· ─╰X─╭●─┤  State
3: ··· ────╰X─┤  State


This shows rotation gates on each qubit [0..3], with 3 values inside the parenthesis to represent the 3 euler angles used for the rotation (qml.Rot(θ, φ, λ)). The entanglers (CNOT gates) are the vertical lines and dot connecting the qubits. For instance, the dot on qubit 0 indicates that it is the control and the target is qubit 1. There are 6 CNOT gates (3 per layer), with rotations done on each qubit before applying CNOT gates. 

## Measure Expectation Value

In [7]:
@qml.qnode(dev)
def qnode_measure(x, params):
    zz_feature_map(x, wires=range(4), reps=1)              #encodes data
    var_ansatz(params)       #transforms encoded state w var ansatz
    #returns expectation values  
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]    



def get_quantum_features(X_mat, params):
    feats = np.array([qnode_measure(x,params) for x in X_mat])
    return feats           #shape [N,n_qubits]

In [8]:
print(qml.draw(qnode_measure)(sample_vec, params))      #now displays zzfeature map  and var ansatz

0: ──H──RZ(-8.78)─╭●─────────────╭●──Rot(-1.03,-1.07,-0.07)────────────────────────────────── ···
1: ──H──RZ(2.52)──╰X──RZ(-11.07)─╰X─╭●────────────────────────────────╭●──Rot(1.89,0.45,0.37) ···
2: ──H──RZ(3.16)────────────────────╰X───────────────────────RZ(3.98)─╰X─╭●────────────────── ···
3: ──H──RZ(1.72)─────────────────────────────────────────────────────────╰X────────────────── ···

0: ··· ─╭●─────────Rot(-0.37,-0.42,0.70)───────────────────────────────────────────────── ···
1: ··· ─╰X─────────────────────────────────────────────────────╭●──Rot(-1.38,-0.25,-0.66) ···
2: ··· ───────────╭●──────────────────────Rot(0.14,0.16,-0.46)─╰X─╭●───────────────────── ···
3: ··· ──RZ(2.71)─╰X──────────────────────Rot(0.99,-1.50,0.19)────╰X───────────────────── ···

0: ··· ─╭●──────────────────────────┤  <Z>
1: ··· ─╰X────────────────────╭●────┤  <Z>
2: ··· ──Rot(-0.18,1.06,0.26)─╰X─╭●─┤  <Z>
3: ··· ──Rot(0.02,0.14,0.79)─────╰X─┤  <Z>


Here you can see the original ZZ featuremap applied to the qubits: each one has a Hadamard gate for placing them into superpositions, then an RZ gate to rotate the qubits around the Z axis by an angle proportionate to its xi value (encodes the data into it's phase), then uses CNOT entanglers and RZ gates to connect qubits. 

The second layer is the Variational Ansatz, where the rotation gates (with the 3 angles) allow for any single-qubit rotation in the bloch sphere, which makes it possible for the ansatz to explore all possible states. 

The final measurement for each qubit is <Z>, which is the measured expectation value of PauliZ operator on each qubit. They then become a classical feature vector we input into the classic models. 

### Training Params



This trains the parameters using the variational ansatz and uses a gradient descent optimizer to help train/optimize the params over 50 epochs (using a MSE loss function). 

This block of code runs for 30+ min, so I have it save the results to the folder /notebooks/params. The bottom of the code will be commented out so it will not run each time. 

In [16]:
# defining a cost function - MSE 
def cost(params, X, y):         #params (layers, nqubits, 3 euler angles), x=classical data [N, nfeatures], y=labels (0 or 1)
    preds = np.array([qnode_measure(x, params) for x in X])   #for each x, runs zzmap, var ansatz, returns expect value of Z for qubit - [N, nfeatures]
    #loss function
    return np.mean((preds[:,0] - y)**2)  # MSE betwen predicted quantum expect values and classic labels y


opt = qml.GradientDescentOptimizer(stepsize=0.1)    #tweaks params 

#training loop
for epoch in range(50):        #optimizes over 50 iterations 
    params = opt.step(lambda p: cost(p, X_train, y_train), params)   #we try to reduce cost while updating params each step 


np.save("params/trained_params.npy", params)    #saves trained numpy array of params to params folder
print(qml.draw(ansatz_preview)(params))   #recheck 


KeyboardInterrupt: 

## Training Model (Classical Classifiers)

### Logistic Regression

In [ ]:
params = np.load("params/trained_params.npy")
#expectation values are treated like classical features 
Xq_train = get_quantum_features(X_train, params)     
Xq_test = get_quantum_features(X_test, params)

q_lg_clf = LogisticRegression().fit(Xq_train, y_train)
q_lg_y_pred = q_lg_clf.predict(Xq_test)  

print("Quantum-feature accuracy:", q_lg_clf.score(Xq_test, y_test))
print("\n\nConfusion Matrix: \n", confusion_matrix(y_test, q_lg_y_pred))    
print("\nClassification Report:\n", classification_report(y_test, q_lg_y_pred))

Quantum-feature accuracy: 0.5314420803782506


Confusion Matrix: 
 [[  24  956]
 [  35 1100]]

Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.02      0.05       980
           1       0.54      0.97      0.69      1135

    accuracy                           0.53      2115
   macro avg       0.47      0.50      0.37      2115
weighted avg       0.48      0.53      0.39      2115



### SVM

In [14]:
q_svm_clf = SVC(kernel='linear', C=1.0, random_state=42)
q_svm_clf.fit(Xq_train, y_train)   

q_svm_y_pred = q_svm_clf.predict(Xq_test)  

print("Quantum-feature accuracy:", q_svm_clf.score(Xq_test, y_test))
print("\n\nConfusion Matrix: \n", confusion_matrix(y_test, q_svm_y_pred))    
print("\nClassification Report:\n", classification_report(y_test, q_svm_y_pred))

Quantum-feature accuracy: 0.5366430260047281


Confusion Matrix: 
 [[   0  980]
 [   0 1135]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.54      1.00      0.70      1135

    accuracy                           0.54      2115
   macro avg       0.27      0.50      0.35      2115
weighted avg       0.29      0.54      0.37      2115



c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\annap\miniconda3\envs\qfm-env\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", res

### Save Models

In [ ]:
joblib.dump(q_lg_clf, "models/quantum_logreg.pkl")
joblib.dump(q_svm_clf, "models/quantum_svm.pkl")